# NanoSafety data summary

- using aggregated search

In [ ]:
import yaml
from solrscope import aa
import ipywidgets as widgets
import requests
from importlib import reload 
from solrscope import client_solr
from solrscope import client_ambit
from solrscope import annotation
import pandas as pd
import numpy as np
import json
import pandas as pd
import qgrid

In [3]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
display(service_widget)

Select enanoMapper aggregated search service:


Dropdown(description='Service:', options=('https://api.ideaconsult.net/enanomapper', 'https://api.ideaconsult.…

In [4]:
if config_security!='':
    print('Enter `{}` you have received upon subscription to http://api.ideaconsult.net'.format(config_security) )
    style = {'description_width': 'initial'}
    apikey_widget=widgets.Text(
        placeholder='',
        description=config_security,
        disabled=False,
        style=style
    )
    display(apikey_widget)

Enter `X-Gravitee-Api-Key` you have received upon subscription to http://api.ideaconsult.net


Text(value='', description='X-Gravitee-Api-Key', placeholder='', style=DescriptionStyle(description_width='ini…

In [5]:
service_uri=service_widget.value
print("Sending queries to {}".format(service_uri))
facets = client_solr.Facets()

Sending queries to https://api.ideaconsult.net/nanoinformatix


In [6]:
    query_widget=widgets.Text(
        placeholder='',
        value="*:*",
        description="Query",
        disabled=False,
        style=style
    )
    display(query_widget)

Text(value='*:*', description='Query', placeholder='', style=DescriptionStyle(description_width='initial'))

In [7]:
#_query="owner_name_s:GRACIOUS"
#_query="DPP_nano"
_query=query_widget.value
print(_query)
method_field="E.method_s"
# method_field="guidance_s"
fields=["topcategory_s","endpointcategory_s",method_field,"substanceType_s","publicname_s","reference_owner_s"]
colnames=["Z"]
colnames.extend(fields)
colnames.append("Number of data points")
print(colnames)
#["Z","Method","Material type","Data provider","Number of data points"]
_stats=[]
reload(client_solr)
def process(prefix,val,count,key,_tuple):
    if len(_tuple)==len(fields):
        _tuple = (*_tuple,val,count)
        _stats.append(_tuple)
        #print("{}".format(_tuple)) 
    
query=facets.getQuery(query=_query,facets=fields,fq="type_s:study")
#print(query)
r = client_solr.post(service_uri,query=query,auth=aa.GraviteeAuth(apikey_widget.value))
response_json=r.json()
if r.status_code==200:
    #print(response_json)
    facets.parse(response_json['facets'],prefix=">",process=process)
    df = pd.DataFrame(_stats,columns=colnames).drop("Z", axis=1)
    a = annotation.DictionarySubstancetypes()
    df[ 'substanceType_s']=df[ 'substanceType_s'].apply(a.annotate)
    a = annotation.DictionaryEndpointCategory()
    df[ 'endpointcategory_term']=df[ 'endpointcategory_s'].apply(a.annotate)
    a = annotation.DictionaryAssays()
    df[ 'method_term']=df[method_field].apply(a.annotate)
    display(df.head())
else:
    print(r.status_code)

*:*
['Z', 'topcategory_s', 'endpointcategory_s', 'E.method_s', 'substanceType_s', 'publicname_s', 'reference_owner_s', 'Number of data points']


,topcategory_s,endpointcategory_s,E.method_s,substanceType_s,publicname_s,reference_owner_s,Number of data points,endpointcategory_term,method_term
0,TOX,TO_GENETIC_IN_VITRO_SECTION,MICRONUCLEUS ASSAY,chemical substance,CONTROL,IPL,330,http://www.bioassayontology.org/bao#BAO_0002167,MICRONUCLEUS ASSAY
1,TOX,TO_GENETIC_IN_VITRO_SECTION,MICRONUCLEUS ASSAY,chemical substance,CONTROL,ANSES,279,http://www.bioassayontology.org/bao#BAO_0002167,MICRONUCLEUS ASSAY
2,TOX,TO_GENETIC_IN_VITRO_SECTION,MICRONUCLEUS ASSAY,chemical substance,CONTROL,INRS (FRANCE),236,http://www.bioassayontology.org/bao#BAO_0002167,MICRONUCLEUS ASSAY
3,TOX,TO_GENETIC_IN_VITRO_SECTION,MICRONUCLEUS ASSAY,chemical substance,CONTROL,INSA,207,http://www.bioassayontology.org/bao#BAO_0002167,MICRONUCLEUS ASSAY
4,TOX,TO_GENETIC_IN_VITRO_SECTION,MICRONUCLEUS ASSAY,chemical substance,CONTROL,FIOH,64,http://www.bioassayontology.org/bao#BAO_0002167,MICRONUCLEUS ASSAY


In [8]:
table = pd.pivot_table(df, values='Number of data points', index=['topcategory_s','endpointcategory_s',method_field], columns=[ 'substanceType_s',"publicname_s"], aggfunc=np.sum).fillna("");
display(table.head())

table.reset_index().to_excel("method_material.xlsx")

substanceType_s                                                      (14)-beta-D-glucan  \
publicname_s                                                        BLEACHED BIRCH PULP   
topcategory_s endpointcategory_s    E.method_s                                            
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                       
                                    _                                                     
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                        
                                    Daphnia magna                                         
                                    OECD TG 201                                           

substanceType_s                                                                     \
publicname_s                                                        UPM BIRCH PULP   
topcategory_s endpointcategory_s    E.method_s                                       
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                  
                                    _                                                
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                   
                                    Daphnia magna                                    
                                    OECD TG 201                                      

substanceType_s                                                                              \
publicname_s                                                        UPM Bleached Birch Pulp   
topcategory_s endpointcategory_s    E.method_s                                                
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                           
                                    _                                                         
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                            
                                    Daphnia magna                                             
                                    OECD TG 201                                               

substanceType_s                                                                   CHEBI_50828\t  \
publicname_s                                                        25 NM (+) GREEN FLUORESCENT   
topcategory_s endpointcategory_s    E.method_s                                                    
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                               
                                    _                                                             
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                
                                    Daphnia magna                                                 
                                    OECD TG 201                                                   

substanceType_s                                                                \
publicname_s                                                        25 NM (-)   
topcategory_s endpointcategory_s    E.method_s                                  
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA             
                                    _                                           
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                              
                                    Daphnia magna                               
                                    OECD TG 201                                 

substanceType_s                                                                                  \
publicname_s                                                        25 NM (-) GREEN FLUORESCENT   
topcategory_s endpointcategory_s    E.method_s                                                    
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA               

```
# try different excel writer
from StyleFrame import StyleFrame, Styler, utils
sf = StyleFrame(table.reset_index())
excel_writer=excel_writer = StyleFrame.ExcelWriter("method_material.xlsx")
sf.to_excel(excel_writer=excel_writer)
excel_writer=excel_writer.save()
```


In [9]:
table = pd.pivot_table(df, values='reference_owner_s', index=['topcategory_s','endpointcategory_s',method_field], columns=[ 'substanceType_s','publicname_s'], 
                             aggfunc=lambda x: ' '.join(str(v) for v in x)).fillna("")

#display(table.head())
display(table)


substanceType_s                                                                                     (14)-beta-D-glucan  \
publicname_s                                                                                       BLEACHED BIRCH PULP   
topcategory_s endpointcategory_s                E.method_s                                                               
ECOTOX        EC_ALGAETOX_SECTION               PSEUDOKIRCHNERIELLA SUBCAPITATA                                          
                                                _                                                                        
              EC_DAPHNIATOX_SECTION             ACUTE TOXICITY                                                           
                                                Daphnia magna                                                            
                                                OECD TG 201                                                              
                                                OECD TG 202                                                              
                                                SOP-Toxicity Test with Daphnia magna (without NOM)                       
                                                _                                                                        
              EC_FISHTOX_SECTION                ALAMAR BLUE                                                              
                                                CFDA-AM                                                                  
                                                NRU                                                                      
                                                _                                                                        
              EC_PLANTTOX_SECTION               _                                                                        
              EC_SEDIMENTDWELLINGTOX_SECTION    _                                                                        
              EC_SOILDWELLINGTOX_SECTION        ISO 10872:2010 Water QualityDetermination of th...                       
                                                SOP-Toxicity Test with C. elegans                                        
              EC_SOIL_MICRO_TOX_SECTION         _                                                                        
P-CHEM        AGGLOMERATION_AGGREGATION_SECTION SAXS                                                                     
                                                SEM                                                                      
              ANALYTICAL_METHODS_SECTION        Chemical composition of particles                                        
                                                EDS                                                                      
                                                IC                                                                       
                                                ICP-MS                                                                   
                                                ICP-OES                                                                  
                                                X-ray Photoelectron Spectroscopy (XPS)                                   
                                                _                                                                        
              ASPECT_RATIO_SHAPE_SECTION        SEM                                                                      
                                                TEM                                                                      
                                                _                                                                        
              CRYSTALLINE_PHASE_SECTION         XRD                                             

In [10]:
table.reset_index().to_excel("method_material_provider.xlsx")